# cg SNP of S. pluranimalium samples

## Running the snippy workflow of bactopia 

Samples sequenced by MinION were provided by Jun m

In [ ]:
First, I create a local copy of the samples and store them in a correct format. 
To be used by bactopia:
    - Each directory must represent a sample by name
    - Sample directories must have a subdirectory called assembly which contains the contigs
    

In [ ]:
# structure for assemblies in bactopia
sample name
-- asemblt
----contigs.fa

# running workflow
... bactopia --wf snippy ...

In [ ]:
cp -r ~/projects/rrg-whsiao-ab/globus_share/minion_data/2022/result_spluranimalium_AHC_2022/combined/5.0.assembly/* \
    /project/6056895/mdprieto/2023_strep_pluranimalium/

# in the target directory, remove all but assembly dir
find ./barcode* -mindepth 1 ! -regex '^./barcode.*/assembly.fasta' -delete

# then, create assembly subdir and put the assembly there compressed
for i in $(ls -d bar*)
    do 
    mkdir $i/assembly
    mv $i/assembly.fasta $i/assembly/$i.fna
    gzip $i/assembly/$i.fna
    done

# and rename the contigs file
    for i in $(ls .)
    do 
    cp $i/assembly/draft_assembly.fasta $i/assembly/$i.fna
    done

In [ ]:
samples="/project/6056895/mdprieto/s_pluranimalium_2023"
reference="/project/6056895/mdprieto/references/GCF_002953735.1_ASM295373v1_genomic.gbff"

In [ ]:
#!/bin/bash
#SBATCH --account=rrg-whsiao-ab
#SBATCH --mem-per-cpu=12G #  GB of memory per cpu core
#SBATCH --time=02:00:00
#SBATCH --ntasks=1 # tasks in parallel / number of cores
#SBATCH --cpus-per-task=8 # number of cores per task
#SBATCH --job-name="s_pluranimalium_20230111"
#SBATCH --chdir=/scratch/mdprieto/
#SBATCH --output=./temp_results/%j_pluranimalium_jan11.out

################################## preparation #########################################

# load singularity and nextflow
module load singularity
module load nextflow

# save paths to directories
samples="/project/6056895/mdprieto/s_pluranimalium_2023"
reference="/project/6056895/mdprieto/references/GCF_002953735.1_ASM295373v1_genomic.gbff"

# define new temp folders for singularity and nextflow
mkdir -p /scratch/$USER/singularity/tmp
export SINGULARITY_CACHEDIR="/project/6007413/cidgoh_share/singularity_imgs"
export SINGULARITY_TMPDIR="/scratch/$USER/singularity/tmp"
export NXF_SINGULARITY_LIBRARYDIR="/project/6007413/cidgoh_share/singularity_imgs"
export SINGULARITYENV_APPEND_PATH=$PATH

################################## nextflow run #########################################

nextflow run bactopia/bactopia -r v2.1.1 -with-singularity bactopia_2.1.1.sif \
    -profile singularity \
    --wf snippy \
    --bactopia $samples \
    --include include.txt \
    --reference $refere nce


#### Note 

Unfortunately bactopia will run `snippy` workflow only on assemblies produced inside the same pipeline. Also, `snippy` is configured to run based on reads and for this exercise, we want to quickly analyze the already assembled/QC genomes. 

Thus, I will explore either running the whole bactopia pipeline from scratch using the raw reads or as a better alternative, run `snippy` outside of bactopia. 

## Try snippy alone

In case snippy needs to be installed, not automatically available in Cedar cluster

In [ ]:
singularity_imgs="/project/6007413/cidgoh_share/singularity_imgs"
cd $singularity_imgs

singularity build snippy_4.6.0.sif https://depot.galaxyproject.org/singularity/snippy%3A4.6.0--hdfd78af_2

### Preparation for snippy run


To analyze multiple sequences, `snippy` requires a tab file where the first column is the sample name and the second column is the `PATH` to the reads or assembly.

To generate the file, we run the following commands in the directory containing the assemblies. 

In [ ]:
# change to genomic data directory
cd /project/6056895/mdprieto/2023_strep_pluranimalium 

# extract all sequences
for i in $(ls -d barcode*)
    do
    gzip -d /project/6056895/mdprieto/2023_strep_pluranimalium/$i/assembly/$i.fna.gz
    done

# paste all info in a file
find /project/6056895/mdprieto/2023_strep_pluranimalium -name 'barcode*' -type f | \
    sort > filenames.tab
ls -d barcode* >> filenames.tab

# use paste and regex to paste as needed
paste <(grep -E '^barcode.*' filenames.tab) \
      <(grep -E '^/project.*' filenames.tab) \
    > filenames_snippy.tab

# create a subset of samples (n=3) for testing in new file
head -n 3 filenames_snippy.tab > trial_snippy.tab

### Testing and running snippy

Establish an interactive session to test how it works in a few samples only

In [ ]:
salloc --time=1:30:0 --cpus-per-task=8 --mem 16G --account=rrg-whsiao-ab

First, we will load the necessary modules (`singularity`) and create environmental variables that will make our coding easier later.


In [3]:
module load singularity
cd ~/scratch

# create environment variables for PATH to genomes, reference, input.tab, and mount drives
singularity_imgs="/project/6007413/cidgoh_share/singularity_imgs"
ref_pluranimailum="/project/6056895/mdprieto/references/GCF_002953735.1_ASM295373v1_genomic.fna"
genomes_dir="/project/6056895/mdprieto/2023_strep_pluranimalium"
BIND_MOUNT="-B /home -B /project -B /scratch -B /localscratch"

# establish cache and tmp for singularity in scratch, project dir is read_only
export SINGULARITY_CACHEDIR="/project/6007413/cidgoh_share/singularity_imgs"
export SINGULARITY_TMPDIR="/scratch/$USER/singularity/tmp"

SyntaxError: invalid syntax (2671485475.py, line 1)

For the `snippy` run, we will call the container from singularity. 

In `snippy-multi` we can produce all the commands necessary to run core SNP from a set of samples with a common reference. 
The results are saved in a script that can be run from the shell. 

**Issues:**
1. The tool did not read the reference in `.gbff` format, so we are adding the files in fasta format `.fna`
2. Not sure why, the fasta files mus be uncompressed
3. Run in scratch, results will be saved in a subfolder

In [ ]:
# for trial run replace input file with trial_snippy.tab

singularity exec $BIND_MOUNT \
    $singularity_imgs/snippy_4.6.0.sif snippy-multi \
    $genomes_dir/filenames_snippy.tab \
    --ref $ref_pluranimailum \
    --cpus 8 \
    --force | \
sed -e 's#^#singularity exec -B /home -B /project -B /scratch -B /localscratch /project/6007413/cidgoh_share/singularity_imgs/snippy_4.6.0.sif #' > /scratch/mdprieto/scripts/pluranimalium_snippy.sh

less /scratch/mdprieto/scripts/pluranimalium_snippy.sh   # check the script makes sense

In [ ]:
Now, we can simply run the script we designed directly on an interactive allocation of as part of a job to the cluster. 
It should return results for every sample inside a specific directory and a summary of the core genome SNP.

In [ ]:
# change dir so the results are placed where we want
cd /scratch/mdprieto/temp_results/s_plurianimalium

sh /scratch/mdprieto/scripts/pluranimalium_snippy.sh 

In [ ]:
singularity exec $BIND_MOUNT $singularity_imgs/snippy_4.6.0.sif \
    snippy-clean_full_aln \
    core.full.aln > clean.full.aln

#### Gubbins

`Gubbins` is a software to eliminate the impact of horizontal gene transfer in bacterial genomes over SNV calling. In this case we have a singularity container with the software and we apply it to the final alignment produced by `snippy` [clean.full.aln]

As our samples had a large number of missing data, we specify a high threshold for filtering and add the prefix 'gubbins' to all output files. 

In [ ]:
cd /scratch/mdprieto/temp_results/s_plurianimalium

singularity exec $BIND_MOUNT $singularity_imgs/depot.galaxyproject.org-singularity-gubbins-3.2.1--py39pl5321h87d955d_0.img \
    run_gubbins.py \
    --filter-percentage 40 \
    -p gubbins \
    clean.full.aln 

#### Calculate SNP distances amomng samples 

After filtering and removing recombination sites, we calculate the SNP distances among all samples available for this analysis using `snp-dists` which is contained in `snippy`. 

In [ ]:
singularity exec $BIND_MOUNT $singularity_imgs/snippy_4.6.0.sif \
    snp-sites -c gubbins.filtered_polymorphic_sites.fasta \
    > clean.core.aln    

#### Produce a tree of SNV distances

We use `IQtree` as in **bactopia** to produce a phylogenetic tree based on core genome SNV distances. 

The tool is available in **Cedar**, so we load it using spider and the apply the tree building algorithm to the results obtained in previous steps.

In [ ]:
module load StdEnv/2020  gcc/9.3.0 iq-tree/2.2.1

iqtree2 -s clean.core.aln

